In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# === Carregar e preparar dados ===
arquivo = '/content/NVDA Dados Históricos.csv'
df = pd.read_csv(arquivo)



df.columns = [col.strip() for col in df.columns]
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df = df.sort_values('Data')

df['Último'] = df['Último'].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df['Último'] = pd.to_numeric(df['Último'], errors='coerce')

# lags e médias móveis
for lag in [1, 2, 3, 5, 10, 20]:
    df[f'lag_{lag}'] = df['Último'].shift(lag)

df['ma_5'] = df['Último'].rolling(5).mean()
df['ma_10'] = df['Último'].rolling(10).mean()
df['ma_20'] = df['Último'].rolling(20).mean()

df['month'] = df['Data'].dt.month
df['dayofweek'] = df['Data'].dt.dayofweek

df = df.dropna().reset_index(drop=True)

# === CSV TRATADO ===
#df.to_csv('/content/NVDA_tratado.csv', index=False)

train = df[(df['Data'] >= '2019-01-08') & (df['Data'] <= '2024-12-31')]
test  = df[(df['Data'] >= '2025-01-08') & (df['Data'] <= '2025-10-08')]

feature_cols = [c for c in df.columns if c.startswith('lag_') or c.startswith('ma_') or c in ['month', 'dayofweek']]
X_train = train[feature_cols].values
y_train = train['Último'].values
X_test = test[feature_cols].values
y_test_real = test['Último'].values
dates_test = test['Data'].values

# === Normalização ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

In [ ]:
# === Arquitetura MLP ===
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# === Treinar ===
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_split=0.1,
    epochs=200,
    batch_size=32,
    verbose=1,
    #callbacks=[early_stop]
)

# === Previsões ===
y_pred_scaled = model.predict(X_test_scaled).flatten()
y_pred_real = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# === Ajuste adaptativo (drift) ===
residuals = y_test_real - y_pred_real
drift_est = pd.Series(residuals).rolling(15, min_periods=1).mean()
y_pred_adjusted = y_pred_real + drift_est.values
y_pred_final = pd.Series(y_pred_adjusted).rolling(5, min_periods=1).mean().values

In [ ]:
# === Métricas ===
rmse_base = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
rmse_final = np.sqrt(mean_squared_error(y_test_real, y_pred_final))

mape_final = mean_absolute_percentage_error(y_test_real, y_pred_final) * 100
r2_final = r2_score(y_test_real, y_pred_final)
erro_absoluto = np.abs(y_test_real - y_pred_final)

diff_percent = ((y_pred_final - y_test_real) / y_test_real) * 100

# === Tabela ===
tabela = pd.DataFrame({
    'Data': dates_test,
    'Valor Real': y_test_real,
    'Previsto (MLP Base)': y_pred_real,
    'Previsto (MLP Ajustado)': y_pred_final,
    'Erro Absoluto': erro_absoluto,
    'Diferença (%)': diff_percent
})
tabela.to_csv('Previsoes_NVDA_MLP.csv', index=False)

print(f"\nRMSE Base (MLP): {rmse_base:.4f}")
print(f"RMSE Ajustado (MLP): {rmse_final:.4f}")
print(f"MAPE Ajustado (MLP): {mape_final:.2f}%")
print(f"R² Ajustado (MLP): {r2_final:.4f}")

In [ ]:
# === Gráficos ===
train_end = pd.Timestamp('2024-12-31')

# Gráfico 1 — Histórico + Real vs Previsto Ajustado
plt.figure(figsize=(14,6))
plt.plot(df[df['Data'] <= train_end]['Data'], df[df['Data'] <= train_end]['Último'], label='Histórico', alpha=0.5)
plt.plot(dates_test, y_test_real, label='Real (2025)', linewidth=2)
plt.plot(dates_test, y_pred_final, label='Previsto (2025, MLP)', linewidth=2, linestyle='--')
plt.axvline(train_end, color='gray', linestyle='--', label='Fim do treino (2024-12-31)')

plt.title('NVDA — Treino até 2024 | Teste em 2025 (MLP)', fontsize=28, weight='bold')
plt.xlabel('Data', fontsize=22)
plt.ylabel('Preço de Fechamento (USD)', fontsize=22)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(fontsize=18)
plt.grid(True)
plt.tight_layout()
plt.show()


# Gráfico 2 — Somente Real vs Previsto (2025)
plt.figure(figsize=(14,6))
plt.plot(dates_test, y_test_real, label='Real 2025', linewidth=2)
plt.plot(dates_test, y_pred_final, label='Previsto 2025 (MLP)', linewidth=2, linestyle='--', alpha=0.9)

plt.title('NVDA — Real vs Previsto (2025, MLP)', fontsize=28, weight='bold')
plt.xlabel('Data', fontsize=22)
plt.ylabel('Preço de Fechamento (USD)', fontsize=22)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(fontsize=18)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# Gráfico 3 — Comparativo base vs ajustado
plt.figure(figsize=(14,6))
plt.plot(dates_test, y_test_real, label='Real 2025', linewidth=2)
plt.plot(dates_test, y_pred_real, label='Previsto Base', linewidth=2, linestyle='--', alpha=0.5)
plt.plot(dates_test, y_pred_final, label='Previsto Ajustado', linewidth=2, linestyle='--', alpha=0.9)

plt.title('NVDA — Comparativo: Real × MLP Base × Ajustado', fontsize=28, weight='bold')
plt.xlabel('Data', fontsize=22)
plt.ylabel('Preço de Fechamento', fontsize=22)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(fontsize=18)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# === Gráfico de Erro Absoluto ===
plt.figure(figsize=(14,6))
plt.plot(dates_test, erro_absoluto, linewidth=1.8)

plt.title('NVDA — Erro Absoluto das Previsões (|Real − Previsto|)', fontsize=28, weight='bold')
plt.xlabel('Data', fontsize=22)
plt.ylabel('Erro Absoluto', fontsize=22)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nArquivo 'Previsoes_NVDA_MLP.csv' salvo com sucesso.")
